<a href="https://colab.research.google.com/github/KizunaAE/GColabSD/blob/main/KizunaAE-SD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **StableDiffusion InvokeAI Base Cloud version**



_You don't need additional Google Drive storage because uploaded models are not stored on your Google Drive. After the session ends, all data will be deleted._

In [ ]:
#@markdown # **STEP 1**
#@markdown ## Requirements
#@markdown It might finished with error but is not the error, just execute the next cell

from IPython.display import clear_output

%cd /content
!git clone https://github.com/KizunaAE/InvokeAI
!pip install -q dependency_injector diffusers einops eventlet facexlib flask_cors flask_socketio flaskwebgui getpass_asterisk huggingface-hub
!pip install -q kornia omegaconf pudb pyreadline3 pytorch-lightning realesrgan streamlit taming-transformers-rom1504 test-tube torch-fidelity
!pip install -q torchmetrics transformers picklescan
!pip install -q pillow xformers==0.0.22 triton==2.0.0 -U
clear_output()

!pip install -q git+https://github.com/invoke-ai/GFPGAN@basicsr-1.4.2#egg=gfpgan
!pip install -q git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -q git+https://github.com/Birch-san/k-diffusion.git@mps#egg=k-diffusion
!pip install -q git+https://github.com/invoke-ai/clipseg.git@relaxed-python-requirement#egg=clipseg
!pip install -q git+https://github.com/invoke-ai/PyPatchMatch@0.1.4#egg=pypatchmatch
%cd /content/InvokeAI/
!pip install -q -e .
clear_output()


!wget https://raw.githubusercontent.com/KizunaAE/InvokeAI-colab/main/INITIAL_MODELS.yaml -O /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml
clear_output()

print('[1;32mDone!')

!pip install python-socketio==5.9.0
clear_output()

#exit()

In [ ]:
#@markdown # **STEP 2**
#@markdown ## Downloading models _(checkpoints, LoRAs, ControlNets, etc.)_
#@markdown To configure the downloading of models, edit this file:
#@markdown _/content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml_

#@markdown P.S. It's fully explained in the tutorial.
from IPython.display import clear_output

%cd /content/InvokeAI/
!python /content/InvokeAI/scripts/invokeai-model-install.py --root_dir /content/db --yes

clear_output()
print('[1;32mDone! All models downloaded successfully 🙃')


In [ ]:
#@markdown # **STEP 3**
#@markdown ## Run StableDiffusion InvokeAI

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union
clear_output()


id_rsa_file = "/content/InvokeAI/id_rsa"
id_rsa_pub_file = "/content/InvokeAI/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)
clear_output()

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=False)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
gen_key(ssh_path)
clear_output()

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/InvokeAI/
!python /content/InvokeAI/scripts/invokeai-web.py --root /content/db

/content/InvokeAI
http (80)
http://ofrjqcdimnpayvkfa25qet7wampjljkup5wudiyxqygsbs3hd2qq.remote.moe/

$
 
2024-01-02 03:19:07.449405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 03:19:07.449519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 03:19:07.537949: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 03:19:09.798476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functiona